# PriceTrack: Unlocking Car Market Insights

PriceTrack is a data science project designed to predict the valuation of second-hand cars based on key input parameters. 
Leveraging Linear regression model, it provides data-driven insights to help buyers and sellers make informed decisions.

We will be using some common Python libraries, such as pandas, numpy, matplotlib and statsmodel.

In [537]:
import pandas as pd

## Importing Dataset

We used the Quikr Car data set that comes from the (https://quikr.com) to carry out Exploratory Data Analysis.

The dataset includes 892 cars (rows) and 6 features (columns).

In [538]:
df=pd.read_csv("quikr_car.csv")
print(df.shape)
df.head()

(892, 6)


,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


### Understanding the type of data we're dealing with

Starting with datatypes

In [539]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        892 non-null    object
 1   company     892 non-null    object
 2   year        892 non-null    object
 3   Price       892 non-null    object
 4   kms_driven  840 non-null    object
 5   fuel_type   837 non-null    object
dtypes: object(6)
memory usage: 41.9+ KB


### 📊 Data Types and Measurement Categories  

| Column Name  | Category  | Description |
|-------------|-----------|-------------|
| `name`      | **Nominal**  | Car model names, which are unique identifiers with no inherent order. |
| `company`   | **Nominal**  | Car brands, representing categorical labels with no ranking. |
| `year`      | **Ordinal**  | Represents the manufacturing year; while numerical, it follows a meaningful order but lacks equal intervals. |
| `Price`     | **Ratio**    | Car price, a continuous numerical variable with a meaningful zero point. |
| `kms_driven` | **Ratio**    | Distance traveled by the car, measured in kilometers with a true zero value. |
| `fuel_type` | **Nominal**  | Categorical variable indicating fuel type (e.g., Petrol, Diesel, etc.) with no ranking. |


## Data Preprocessing

#### Replacing wrong values with N/A

In [540]:
df.isna().sum()

name           0
company        0
year           0
Price          0
kms_driven    52
fuel_type     55
dtype: int64

We can see that **kms_driven** and **fuel_type** column has ~50 missing values

In [541]:
preprocessed_df = df.copy()

#### a) Year Column

Finding unique year values

In [542]:
print("Unique years: ", df["year"].unique())

Unique years:  ['2007' '2006' '2018' '2014' '2015' '2012' '2013' '2016' '2010' '2017'
 '2008' '2011' '2019' '2009' '2005' '2000' '...' '150k' 'TOUR' '2003'
 'r 15' '2004' 'Zest' '/-Rs' 'sale' '1995' 'ara)' '2002' 'SELL' '2001'
 'tion' 'odel' '2 bs' 'arry' 'Eon' 'o...' 'ture' 'emi' 'car' 'able' 'no.'
 'd...' 'SALE' 'digo' 'sell' 'd Ex' 'n...' 'e...' 'D...' ', Ac' 'go .'
 'k...' 'o c4' 'zire' 'cent' 'Sumo' 'cab' 't xe' 'EV2' 'r...' 'zest']


We will remove all the invalid year values and convert it to \<NA\>

In [543]:
preprocessed_df["year"] = pd.to_numeric(df["year"], errors='coerce').astype("Int64")
unique_years = preprocessed_df["year"].sort_values().unique()
print("Unique years: ", unique_years)

Unique years:  <IntegerArray>
[1995, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, <NA>]
Length: 22, dtype: Int64


#### b) Km Driven Column

We will convert Kilometers to Integer

In [544]:
kms_series = pd.to_numeric(df["kms_driven"].str.replace(r"\D", "", regex=True), errors="coerce").astype("Int64")

# Finding Extreme Range
extremeKms = (kms_series < 1000) | (kms_series > 400000)
print(f"{extremeKms.sum()} records with extreme kms will be replaced with mean")
kms_series[extremeKms] = pd.NA # will be replaced by mean in later blocks

preprocessed_df["kms_driven"] = kms_series
preprocessed_df["kms_driven"].describe()

21 records with extreme kms will be replaced with mean


count           817.0
mean     47190.228886
std       34104.70073
min            1000.0
25%           28000.0
50%           41800.0
75%           58000.0
max          400000.0
Name: kms_driven, dtype: Float64

Let's see which record has the maximum kms_driven

In [545]:
max_kmsdriven_mask = preprocessed_df["kms_driven"] == preprocessed_df["kms_driven"].max()
max_kmsdriven_row = preprocessed_df[max_kmsdriven_mask]
max_kmsdriven_row

,name,company,year,Price,kms_driven,fuel_type
362,Honda City 1.5 V MT Exclusive,Honda,2010,"2,40,000",400000,Petrol


In [546]:
preprocessed_df[preprocessed_df["year"] == 2010].sort_values(by="kms_driven", ascending=False).head(5)

,name,company,year,Price,kms_driven,fuel_type
362,Honda City 1.5 V MT Exclusive,Honda,2010,"2,40,000",400000,Petrol
430,Toyota Fortuner 3.0 4x4 MT,Toyota,2010,"9,40,000",131000,Diesel
431,Tata Manza ELAN Quadrajet,Tata,2010,"1,55,555",111111,Petrol
783,Tata Sumo Grande MKII GX,Tata,2010,"1,30,000",90000,Diesel
646,Maruti Suzuki Swift VDi ABS,Maruti,2010,"1,90,000",74000,Diesel


If we compare with other cars from the same manufacture year, we can see that the 2nd highest kms_driven is 131,000.

From this, it is clear that there was some wrong input in case of Honda City and kms_driven should have been 40,000.

In [547]:
preprocessed_df.at[preprocessed_df[max_kmsdriven_mask].index[0], 'kms_driven'] = 40000
preprocessed_df[max_kmsdriven_mask]

,name,company,year,Price,kms_driven,fuel_type
362,Honda City 1.5 V MT Exclusive,Honda,2010,"2,40,000",40000,Petrol


#### c) Price Column

Converting Price to Integer

In [548]:
price_series = pd.to_numeric(df["Price"].str.replace(r"\D", "", regex=True), errors="coerce").astype("Int64")

preprocessed_df["Price"] = price_series
preprocessed_df["Price"].describe()

count            857.0
mean     404688.534422
std      465536.544629
min            30000.0
25%           175000.0
50%           299999.0
75%           485000.0
max          8500003.0
Name: Price, dtype: Float64

Let's see which record has the maximum price

In [549]:
max_price_mask = preprocessed_df["Price"] == preprocessed_df["Price"].max()
max_price_row = preprocessed_df[max_price_mask]
max_price_row

,name,company,year,Price,kms_driven,fuel_type
562,Mahindra XUV500 W6,Mahindra,2014,8500003,45000,Diesel


It seems that there was a typing error and the price should have been 8,50,000 instead

In [550]:
preprocessed_df.at[preprocessed_df[max_price_mask].index[0], 'Price'] = 8_50_000
preprocessed_df[max_price_mask]

,name,company,year,Price,kms_driven,fuel_type
562,Mahindra XUV500 W6,Mahindra,2014,850000,45000,Diesel


#### d) Fuel Type Column

Converting Fuel Type to category

In [551]:
preprocessed_df["fuel_type"] = df["fuel_type"].astype("category")
preprocessed_df["fuel_type"].value_counts()

fuel_type
Petrol    440
Diesel    395
LPG         2
Name: count, dtype: int64

In [552]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 892 entries, 0 to 891
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   name        892 non-null    object  
 1   company     892 non-null    object  
 2   year        842 non-null    Int64   
 3   Price       857 non-null    Int64   
 4   kms_driven  817 non-null    Int64   
 5   fuel_type   837 non-null    category
dtypes: Int64(3), category(1), object(2)
memory usage: 38.6+ KB


In [553]:
preprocessed_df[["Price", "kms_driven"]].describe()

,Price,kms_driven
count,857.0,817.0
mean,395762.043174,46749.592411
std,374589.927179,31787.686352
min,30000.0,1000.0
25%,175000.0,28000.0
50%,299999.0,41000.0
75%,485000.0,57923.0
max,3100000.0,200000.0


## Data Cleaning

#### Handling Missing values

In [554]:
preprocessed_df.isna().sum()

name           0
company        0
year          50
Price         35
kms_driven    75
fuel_type     55
dtype: int64

In [555]:
nokms_mask = preprocessed_df["kms_driven"].isna()
noyear_mask = preprocessed_df["year"].isna()
noprice_mask = preprocessed_df["Price"].isna()
nofuel_mask = preprocessed_df["fuel_type"].isna()

temp_df = preprocessed_df[((nokms_mask) & (noyear_mask) & (noprice_mask) & (nofuel_mask))].copy()
print(f"{temp_df.shape[0]} rows are deleted because they don't have any of kms_driven, year, price or fuel_type")
df_without_4_missing = preprocessed_df.drop(axis="index", index=temp_df.index)
print("Remaining dataset:")

df_without_4_missing.isna().sum()

12 rows are deleted because they don't have any of kms_driven, year, price or fuel_type
Remaining dataset:


name           0
company        0
year          38
Price         23
kms_driven    63
fuel_type     43
dtype: int64

Removing Rows with Invalid names

In [556]:
invalid_name_mask = df_without_4_missing["name"].str.len()<5
print("Removed", df_without_4_missing[invalid_name_mask].shape[0], "rows with invalid name")
df_with_valid_names = df_without_4_missing.drop(axis="index", index=df_without_4_missing[invalid_name_mask].index)
df_with_valid_names.shape

Removed 3 rows with invalid name


(877, 6)

Converting LPG to Petrol and dropping \<NA\> fuel type records

In [557]:
def show_distribution(df, column_name):
    value_counts_df = df[column_name].value_counts().reset_index()
    value_counts_df.columns = [column_name.capitalize(), "Count"]
    value_counts_df["Relative"] = (df[column_name].value_counts(normalize=True).values * 100).round(2).astype("str") + "%"
    print(value_counts_df)

    empty_records = df[column_name].isna().sum()
    total_records = df.shape[0]
    
    print("Empty records:",empty_records, end="")
    if empty_records > 0:
        print(f" ({empty_records / total_records:.2f}%)")

In [558]:
df_with_2_fuels = df_with_valid_names.copy()

# Before
print("Before: Had 3 categories with 40 empty records")
show_distribution(df_with_2_fuels, "fuel_type")

# Converting LPG to Petrol
df_with_2_fuels.loc[df_with_2_fuels["fuel_type"]=="LPG", "fuel_type"] = "Petrol"

# Dropping <NA> records
df_with_2_fuels.dropna(subset=["fuel_type"], inplace=True)
df_with_2_fuels["fuel_type"] = df_with_2_fuels["fuel_type"].cat.remove_unused_categories()

# After
print("\nAfter: Converted LPG to Petrol, dropped empty records")
show_distribution(df_with_2_fuels, "fuel_type")

df_with_2_fuels.isna().sum()

Before: Had 3 categories with 40 empty records
  Fuel_type  Count Relative
0    Petrol    440   52.57%
1    Diesel    395   47.19%
2       LPG      2    0.24%
Empty records: 40 (0.05%)

After: Converted LPG to Petrol, dropped empty records
  Fuel_type  Count Relative
0    Petrol    442   52.81%
1    Diesel    395   47.19%
Empty records: 0

name           0
company        0
year           0
Price         21
kms_driven    21
fuel_type      0
dtype: int64

Finding the average total kilometers driven based on **fuel type**

In [559]:
mean_km = df_with_2_fuels.groupby("fuel_type", observed=False)["kms_driven"].mean().to_dict()
mean_km

{'Diesel': 57442.33769633508, 'Petrol': 37385.81566820276}

Finding Mean Annual Kilometers driven based on **fuel_type**

In [560]:
calc_df = df_with_2_fuels.copy()

# Age
base_year = unique_years.max()
calc_df["Age"] = base_year - calc_df["year"] + 1
useful_df = calc_df.loc[(df_with_2_fuels["year"].notna()) & (df_with_2_fuels["kms_driven"].notna())]

# Annual_Km_Driven
calc_df["Annual_Km_Driven"] = useful_df.loc[:,"kms_driven"] / useful_df.loc[:,"Age"]
mean_annual_km = calc_df[~calc_df["Annual_Km_Driven"].isna()].groupby("fuel_type", observed=False)["Annual_Km_Driven"].mean().to_dict()

# Fill Empty Annual_Km_Driven
for fuel_type, mean_km in mean_annual_km.items():
    mask = (calc_df["Annual_Km_Driven"].isna()) & (calc_df["fuel_type"] == fuel_type)
    calc_df.loc[mask, "Annual_Km_Driven"] = mean_km

# Mean Annual Kilometers Driven
mean_annual_km

{'Diesel': 10011.436886549187, 'Petrol': 5080.053693088486}

In [561]:
# Fill missing values in kms_driven using mean_annual_km's respective fuel_type * Age
for fuel_type, mean_km in mean_annual_km.items():
    mask = (calc_df["kms_driven"].isna()) & (calc_df["fuel_type"] == fuel_type)
    calc_df.loc[mask, "kms_driven"] = (calc_df.loc[mask, "Age"] * mean_km).astype(int)
# (calc_df["kms_driven"].isna()) & (calc_df["fuel_type"] == "Diesel")

We dropped the records where price was empty

In [562]:
print("No. of records with missing price:", calc_df["Price"].isna().sum())
filled_df = calc_df.dropna(subset=["Price"])
print("Shape of filled dataset:", filled_df.shape)
filled_df.isna().sum()

No. of records with missing price: 21
Shape of filled dataset: (816, 8)


name                0
company             0
year                0
Price               0
kms_driven          0
fuel_type           0
Age                 0
Annual_Km_Driven    0
dtype: int64

Revisiting the data types after all the cleaning

In [563]:
filled_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 816 entries, 0 to 889
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              816 non-null    object  
 1   company           816 non-null    object  
 2   year              816 non-null    Int64   
 3   Price             816 non-null    Int64   
 4   kms_driven        816 non-null    Int64   
 5   fuel_type         816 non-null    category
 6   Age               816 non-null    Int64   
 7   Annual_Km_Driven  816 non-null    Float64 
dtypes: Float64(1), Int64(4), category(1), object(2)
memory usage: 55.9+ KB


Storing the cleaned data into a CSV file

In [564]:
filled_df.to_csv("cleaned_data.csv", index=False)
filled_df

,name,company,year,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,80000,45000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,2006,425000,140160,Diesel,14,10011.436887
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,325000,28000,Petrol,6,4666.666667
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,575000,36000,Diesel,6,6000.0
6,Ford Figo,Ford,2012,175000,41000,Diesel,8,5125.0
...,...,...,...,...,...,...,...,...
883,Maruti Suzuki Ritz VXI ABS,Maruti,2011,270000,50000,Petrol,9,5555.555556
885,Tata Indica V2 DLE BS III,Tata,2009,110000,30000,Diesel,11,2727.272727
886,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol,11,12000.0
888,Tata Zest XM Diesel,Tata,2018,260000,27000,Diesel,2,13500.0


### Next step: Exploratory Data Analysis (EDA)

We'll look at some visualisations to better understand the data